In [62]:
#importing required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
from torch.autograd import Variable

[!img]segnet.png

In [99]:
class SegNet(nn.Module):
    def __init__(self):
        super(SegNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64,64, kernel_size=3, padding=1)
                
        self.conv3 = nn.Conv2d(64,128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128,128, kernel_size=3, padding=1)
        
        self.conv5 = nn.Conv2d(128,256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256,256, kernel_size=3, padding=1)
        
        self.conv7 = nn.Conv2d(256,512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        
        self.conv8 = nn.Conv2d(512,512, kernel_size=3, padding=1)        
        
        self.upSample = nn.MaxUnpool2d(2, stride=2)
      
        self.decode1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.decode2 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.dec_bn1 = nn.BatchNorm2d(512)
        self.dec_relu = nn.ReLU(inplace=True)
        
        self.dec_bn2 = nn.BatchNorm2d(256)
        
        self.decode3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.decode4 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.dec_bn3 = nn.BatchNorm2d(128)
        
        self.decode5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.decode6 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.dec_bn4 = nn.BatchNorm2d(64)
        self.decode7 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.decode8 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        
        
    def forward(self, x):
        out= self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn1(out)
        out = self.relu(out)
        x1p, id1 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.conv3(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv4(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.maxPool(out)
        x2p, id2 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
                
        out = self.conv5(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv6(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv6(out)
        out = self.bn3(out)
        out = self.relu(out)
        x3p, id3 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.conv7(out)
        out = self.bn4(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn4(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn4(out)
        out = self.relu(out)
        x4p, id4 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.conv8(out)
        out = self.bn5(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn5(out)
        out = self.relu(out)
        out = self.conv8(out)
        out = self.bn5(out)
        out = self.relu(out)
        x5p, id5 = F.max_pool2d(out,kernel_size=2, stride=2,return_indices=True)
        
        out = self.upSample(x5p, id5)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id4)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode1(out)
        out = self.dec_bn1(out)
        out = self.dec_relu(out)
        out = self.decode2(out)
        out = self.dec_bn2(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id3)
        out = self.decode3(out)
        out = self.dec_bn2(out)
        out = self.dec_relu(out)
        out = self.decode3(out)
        out = self.dec_bn2(out)
        out = self.dec_relu(out)
        out = self.decode4(out)
        out = self.dec_bn3(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id2)
        out = self.decode5(out)
        out = self.dec_bn3(out)
        out = self.dec_relu(out)
        out = self.decode6(out)
        out = self.dec_bn4(out)
        out = self.dec_relu(out)
        
        out = self.upSample(out, id1)
        out = self.decode7(out)
        out = self.dec_bn4(out)
        out = self.dec_relu(out)
        out = self.decode8(out)

        return out

In [100]:
model = SegNet()

In [103]:
model

SegNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv8): Conv2d(512, 512, kernel_size=(3, 3), stride=(

In [112]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

import numpy as np
import tqdm

In [113]:
model.train()

# update learning rate
lr = 0.001
for param_group in optimizer.param_groups:
    param_group['lr'] = lr

# define a weighted loss (0 weight for 0 label)
weights_list = [0]+[1 for i in range(17)]
weights = np.asarray(weights_list)
weigthtorch = torch.Tensor(weights_list)
loss = nn.CrossEntropyLoss(weight=weigthtorch).cuda()



total_loss = 0

# iteration over the batches
batches = []
for batch_idx,batch_files in enumerate(tqdm(batches)):

    # containers
    batch = np.zeros((args.batch_size,input_nbr, imsize, imsize), dtype=float)
    batch_labels = np.zeros((args.batch_size,imsize, imsize), dtype=int)

    # fill the batch
    # ...

    batch_th = Variable(torch.Tensor(batch))
    target_th = Variable(torch.LongTensor(batch_labels))

    batch_th =batch_th.cuda()
    target_th = target_th.cuda()

    # initilize gradients
    optimizer.zero_grad()

    # predictions
    output = model(batch_th)

    # Loss
    output = output.view(output.size(0),output.size(1), -1)
    output = torch.transpose(output,1,2).contiguous()
    output = output.view(-1,output.size(2))
    target = target.view(-1)

    l_ = loss(output.cuda(), target)
    total_loss += l_.cpu().data.numpy()
    l_.cuda()
    l_.backward()
    optimizer.step()

TypeError: 'module' object is not callable

In [80]:
a= torch.rand([3,64,64])

In [82]:
c,b =F.max_pool2d(a,kernel_size=2, stride=2,return_indices=True)
print(c.shape, b.shape)

torch.Size([3, 32, 32]) torch.Size([3, 32, 32])


In [90]:
pool = nn.MaxPool2d(2, stride=2, return_indices=True)

uppool = nn.MaxUnpool2d(2, stride=2)

a = torch.tensor([[[[ 1.,  2,  3,  4],
                            [ 5,  6,  7,  8],
                            [ 9, 10, 11, 12],
                            [13, 14, 15, 16]]]])

print(a.shape)
output, indices = pool(a)
print(output.shape, indices.shape)
d=unpool(output, indices)
print(d.shape)

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 2, 2]) torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 4, 4])


### Ongoing